(This notebook helped to build the script, it is not formatted and not clean)

## Build lambda for RDS datalake for Weather data (historical)

### 1. Prompt data from RDS, Wildfires in California

In [1]:
import psycopg2
import pandas as pd

In [80]:
try: 
    conn = psycopg2.connect("host=firedb1.cscdg7rb1snc.us-east-1.rds.amazonaws.com dbname=FIREDB1 user=maren password=0123456789")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [81]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
# Auto commit is very important
conn.set_session(autocommit=True)

In [82]:
try: 
    cur.execute("SELECT FireDiscoveryDateTime, InitialLatitude, InitialLongitude FROM Firedata_Total;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
data=[]
while row:
    data.append(list(row))
    row = cur.fetchone()
data_fire=data
header=['date','lat','long']
df=pd.DataFrame(data, columns=header)
df

,date,lat,long
0,2016-05-05,38.767220,-119.8167
1,2015-07-05,41.421398,-122.4981
2,2019-08-10,41.046330,-122.0796
3,2017-08-07,41.357780,-120.9694
4,2017-08-06,37.751220,-119.1410
...,...,...,...
1456,2021-08-10,34.172500,-116.8266
1457,2021-08-30,33.829170,-116.7631
1458,2021-08-30,34.247230,-116.9277
1459,2021-08-30,34.201170,-116.7913


In [83]:
cur.close()
conn.close()

### 2. Prompt historical data from Weather API

In [7]:
import urllib.request
import json
import pprint
from datetime import date, datetime, timedelta

In [8]:
# We actually don't need the pandas DataFrame, we just need the data insinde, and nested list is much more easier!
request_data=data

In [29]:

pd.to_datetime(df['date'])

0      2016-05-05
1      2015-07-05
2      2019-08-10
3      2017-08-07
4      2017-08-06
          ...    
1456   2021-08-10
1457   2021-08-30
1458   2021-08-30
1459   2021-08-30
1460   2021-09-10
Name: date, Length: 1461, dtype: datetime64[ns]

In [46]:
j1=datetime.strptime('2021-06-10','%Y-%m-%d').date()
j2=datetime.strptime('2021-10-10','%Y-%m-%d').date()
print(type(j1))

<class 'datetime.date'>


In [47]:
new_df=df[df.date.between(j1,j2)]

In [53]:
i=2
print('row %i not imported'%i)

row 2 not imported


In [49]:
del new_df

In [58]:
today=date.today()
month_ago=today-timedelta(days=30)
print(type(today))
print(type(month_ago))

<class 'datetime.date'>
<class 'datetime.date'>


In [68]:
def API_query_history(loc, date, delta, apikey='Z3Z4Q9PHE3MSXWKUPQNQ8434D', unit='metric', agg='24'):
    # This is the core of our weather query URL
    BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history'

    ApiKey=apikey
    #UnitGroup sets the units of the output - us or metric
    UnitGroup=unit

    #Locations for the weather data. Multiple locations separated by pipe (|)
    Locations=loc

    #1=hourly, 24=daily
    AggregateHours=agg

    #Params for history only
    date_of_interest=date
    StartDate = str(date_of_interest+timedelta(days=-delta))
    EndDate= str(date_of_interest+timedelta(days=delta))

    # Set up the query
    QueryParams = '?aggregateHours=' + AggregateHours + '&startDateTime=' + StartDate + '&endDateTime='+ EndDate + '&unitGroup=' + UnitGroup + '&shortColumnNames=true'

    Locations='&locations=' + Locations

    ApiKey='&key='+ApiKey

    # Build the entire query
    URL = BaseURL + QueryParams + Locations + ApiKey+"&contentType=json"

    return URL

#building header, only need to loop once
def json_header_extractor():
    dict_locations=weatherData['locations']
    key_location=list(weatherData['locations'].keys())
    header=[]
    for key in dict_locations[str(key_location[0])].keys():
        if key=='values':
            header.extend(list(dict_locations[str(key_location[0])][key][0].keys()))
        else:
            header.append(key)
    return header

#building data, in the 'values' section, there is the data for the forecasted days, so if forecstDays=3, there is 3 elements in the list 'values'
def json_data_extractor():
    dict_locations=weatherData['locations']
    key_location=list(weatherData['locations'].keys())
    all_data=[]
    for i in range((delta*2)+1):
        data=[]
        for key, value in dict_locations[str(key_location[0])].items():
            if key=='values':
                try:
                    data.extend(list(dict_locations[str(key_location[0])][key][i].values()))
                except:
                    pass
            else:
                data.append(value)
        all_data.append(data)
    return all_data

In [ ]:
n=0
delta=2
raw_data=[]
for fire in request_data:
    date=fire[0]
    lat=fire[1]
    long=fire[2]
    loc=str(lat)+','+str(long)
    URL=API_query_history(loc, date, delta)
    print(URL)
    response = urllib.request.urlopen(URL)
    data = response.read()
    weatherData = json.loads(data.decode('utf-8'))
    if n==0:
        header=json_header_extractor()
        data=json_data_extractor()
        raw_data.extend(data)
        n+=1
    else:
        data=json_data_extractor()
        raw_data.extend(data)

In [28]:
#Build DataFrame
df=pd.DataFrame(raw_data,columns=header)
df_manip=df

#Quick manipulation in order to facilitate the insertion  in the RDS tables (delete columns with no data, and re-arrange column order)
# delete 'stationContribution', 'info', 'index', 'distance', 'time', 'currentConditions' , 'alerts' because they are empty columns -> no data
df_manip=df_manip.drop(['stationContributions', 'datetime','info', 'index', 'distance', 'time', 'currentConditions' , 'alerts'], axis=1)

# re-arrange column
re_arrg_col=['tz','longitude','latitude','name','address','id']
for col in re_arrg_col:
    shiftpos=df_manip.pop(col)
    df_manip.insert(0,col,shiftpos)

#Save the DataFrame to .csv
new_df=df_manip
new_df.to_csv('weather_API_historical_data.csv', index=False)


### 4. Load data in RDS datalake

In [35]:
import psycopg2
import os
import pprint

In [34]:
ENDPOINT = os.environ['ENDPOINT']
DB_NAME = os.environ['DB_NAME']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

KeyError: 'ENDPOINT'

In [24]:
try: 
    #print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    #conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host=datalake.cbqjcib7k7og.us-east-1.rds.amazonaws.com dbname=datalake_weather user=nathan password=123456789")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [25]:
try: ##
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
# Auto commit is very important
conn.set_session(autocommit=True)

In [45]:
#Create table for historical data
cur.execute("CREATE TABLE IF NOT EXISTS HISTORICAL_WEATHER_DATA (id VARCHAR(70), address VARCHAR(80), name VARCHAR(70), latitude FLOAT, longitude FLOAT, tz VARCHAR(30), wdir FLOAT, temp FLOAT, maxt FLOAT, visibility FLOAT, wspd FLOAT, datetimeStr DATE, solarenergy FLOAT, heatindex FLOAT, cloudcover FLOAT, mint FLOAT, precip FLOAT, solarradiation FLOAT, weathertype VARCHAR(120),	snowdepth FLOAT, sealevelpressure FLOAT, snow FLOAT, dew FLOAT, humidity FLOAT, precipcover FLOAT, wgust FLOAT, conditions VARCHAR(90), windchill FLOAT);")

In [ ]:
cur.execute("TRUNCATE TABLE HISTORICAL_WEATHER_DATA")

In [51]:
#Upload data in the table
data=new_df

cols = ",".join([str(i) for i in data.columns.tolist()])

imported_rows=0
excluded_rows_index=[]
for i,row in data.iterrows():
    try:
        sql = "INSERT INTO HISTORICAL_WEATHER_DATA  (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cur.execute(sql, tuple(row))
        conn.commit()
        imported_rows+=1
    except:
        excluded_rows_index.append(i)
        print(i)
print('Number of imported rows: ',imported_rows)

30
31
32
33
34
115
116
117
118
119
175
176
177
178
179
240
241
242
243
244
515
516
517
518
519
975


In [27]:
cur.execute("select count(*) from HISTORICAL_WEATHER_DATA;")
number_rows=cur.fetchall()[0][0]
print(number_rows)

975


In [28]:
cur.close()
conn.close()